In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

import sys
sys.path.insert(1, '../../../../scripts/')
from s3_support import *

We're going to try modeling against all available data points and see what comes out of it.

# load data

target
- churned orgs

sources
- logs
- segment data points (logins, integrations, cms)
- transaction volume growth
- analytics

sources will need to be aggregated with lag values. try modeling
1. last 3 months of data
2. last 6 months of data
3. last 12 months of data

## churned orgs

In [2]:
# load orgs for churn data
orgs = get_dataframe_from_file("qgiv-stats-data", "organizations.names.csv")

cols = ['id', 'org_name', 'live_date', 'date_closed', 'signup_step_one', 'first_transaction_date',
       'pricing_package', 'segment', 'tags', 'reason_closed', 'additional_churn_info']
orgs = orgs[cols]

# cleanup 
date_cols = ['live_date', 'date_closed', 'signup_step_one', 'first_transaction_date']
for c in date_cols:
    orgs[c] = orgs[c].apply(lambda x: np.nan if x == '12/31/1969' else x)
    orgs[c] = pd.to_datetime(orgs[c])
    
str_cols = ['org_name', 'tags', 'reason_closed', 'additional_churn_info']
for c in str_cols:
    orgs[c] = orgs[c].fillna('')

In [3]:
q = "select id, datecreated from organization"
org_created = redshift_query_read(q, schema="production")
orgs['datecreated'] = orgs['id'].apply(lambda x: org_created[org_created['id']==x]['datecreated'].iloc[0])

In [4]:
orgs['churned'] = ~orgs['date_closed'].isna()

## logs

In [5]:
q = "select * from logs"
logs = redshift_query_read(q)

logs['systemid'] = logs['systemid'].fillna(0).astype('int')
logs['systemtype'] = logs['systemtype'].fillna(0).astype('int')
logs['org'] = logs['org'].fillna(0).astype('int')

len(logs)

3555309

In [6]:
logs = logs[(logs['org']!=0)&(logs['form']!=0)][['org', 'systemtype', 'created']].copy()
logs['month'] = pd.to_datetime(logs['created']).dt.to_period('M')
logs_agg = logs.groupby(['org', 'month', 'systemtype'])['created'].count().reset_index()

In [7]:
agg_data = None
for o in logs_agg['org'].unique():
    this_df = logs_agg[logs_agg['org']==o].copy()
    last_3_months = this_df['month'].sort_values(ascending=True).tail(4)
    this_df = this_df[this_df['month'].isin(last_3_months)]
    
    if agg_data is None:
        agg_data = this_df
    else:
        agg_data = agg_data.append(this_df)
agg_data.columns = ['org', 'month', 'systemtype', 'count']

In [8]:
len(agg_data['systemtype'].unique())

33

In [9]:
agg_data['pct_change'] = agg_data.groupby(['org', 'month', 'systemtype'])['count'].pct_change()
agg_data = agg_data.groupby(['org', 'systemtype'])['pct_change'].mean().reset_index()
logs_pvt = agg_data.pivot(index="org", columns="systemtype", values="pct_change").reset_index().fillna(0)
logs_pvt.columns = ["org"] + ["system_{}".format(c) for c in logs_pvt.columns if c != "org"]

In [10]:
logs_pvt.head()

,org,system_0,system_2,system_4,system_8,system_11,system_12,system_13,system_15,system_18,...,system_39,system_40,system_41,system_42,system_43,system_44,system_46,system_48,system_49,system_50
0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0.0,0.0,0.0,0.0,0.0,1.2,0.0,0.0,0.0,...,-0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## segment data points (logins, integrations, cms)

In [11]:
q = '''select
            users.org as org,
            date_trunc('month', original_timestamp) as month,
            count(login.user_id) as users
        from login
            left join users on login.user_id=users._id
        group by date_trunc('month', original_timestamp), org
        order by date_trunc('month', original_timestamp) desc;'''
logins = redshift_query_read(q, schema="secure")

In [12]:
logins['month'] = pd.to_datetime(logins['month'])
last_3_months = logins['month'].sort_values(ascending=True).unique()[-4:]
# get list of orgs with recent logins
orgs_with_recent_logins = logins[logins['month'].isin(last_3_months)]['org'].unique()

In [13]:
logins['pct_change'] = logins.groupby(['org', 'month'])['users'].pct_change()
# get the growth trend in login activity
login_change = logins.groupby('org')['pct_change'].mean().reset_index()

In [14]:
login_change['recent_login'] = login_change['org'].apply(lambda x: x in orgs_with_recent_logins)
login_change.head()

,org,pct_change,recent_login
0,100355,0.000000,True
1,100356,2.216667,True
2,10165,0.000000,True
3,10168,0.556859,True
4,10172,10.296707,True


In [15]:
q = '''select
            users.org as org,
            date_trunc('month', original_timestamp) as month,
            count(activated_integration.id) as activations 
        from activated_integration
            left join users on activated_integration.uuid=users.uuid
        group by month, org
        order by month desc;'''
integrations = redshift_query_read(q, schema="secure")

In [16]:
integrations['month'] = pd.to_datetime(integrations['month'])
last_3_months = integrations['month'].sort_values(ascending=True).unique()[-4:]
# get list of orgs with recent integration activity
orgs_with_recent_integrations = integrations[integrations['month'].isin(last_3_months)]['org'].unique()

In [17]:
integrations['pct_change'] = integrations.groupby(['org', 'month'])['activations'].pct_change()
# get growth trend in integration activity
integrations_change = integrations.groupby('org')['pct_change'].mean().reset_index()

In [18]:
integrations_change['recent_activation'] = integrations_change['org'].apply(lambda x: x in orgs_with_recent_integrations)
integrations_change.head()

,org,pct_change,recent_activation
0,1100,0.00,True
1,113,-0.50,True
2,131077,-0.75,True
3,1445,0.00,True
4,185347,0.00,True


In [19]:
q = '''select
            users.org as org,
            date_trunc('month', original_timestamp) as month,
            count(saved_page.id) as pages_saved
        from saved_page
            left join users on saved_page.uuid=users.uuid
        group by month, org
        order by month desc;'''
cms = redshift_query_read(q, schema="secure")

In [20]:
cms['month'] = pd.to_datetime(cms['month'])
last_3_months = cms['month'].sort_values(ascending=True).unique()[-4:]
# get list of orgs with recent cms activity
orgs_with_recent_cms = cms[cms['month'].isin(last_3_months)]['org'].unique()

In [21]:
cms['pct_change'] = cms.groupby(['org', 'month'])['pages_saved'].pct_change()
# get growth trend in cms activity
cms_change = cms.groupby('org')['pct_change'].mean().reset_index()

In [22]:
cms_change['recent_activity'] = cms_change['org'].apply(lambda x: x in orgs_with_recent_cms)
cms_change.head()

,org,pct_change,recent_activity
0,0,-0.50,True
1,100355,-0.75,True
2,100356,0.00,False
3,1005,-0.75,True
4,10160,-0.50,True


## transaction & active form growth

In [23]:
# query transactions grouped by month
q = '''select
            org,
            date_trunc('quarter', date) as quarter,
            count(distinct form) as forms,
            sum(amount) as volume
        from transactions
            where status='A'
            group by org, date_trunc('quarter', date);'''
trans = redshift_query_read(q)

trans['quarter'] = pd.to_datetime(trans['quarter'])
trans = trans.sort_values('quarter', ascending=True)

# drop 0 org from agg
trans = trans[trans['org']!=0]

In [24]:
# group trans by org and calculate exploratory stats on volume
agg_trans = trans[trans['volume']>0].groupby('org')[['volume', 'forms']].agg({'max', 'mean', 'last'}).reset_index()
agg_trans.columns = ['org', 'volume_max', 'volume_mean', 'volume_last', 'forms_max', 'forms_mean', 'forms_last']

agg_trans['volume_max_last_diff'] = (agg_trans['volume_last'] - agg_trans['volume_max']) / agg_trans['volume_max']
agg_trans['volume_mean_last_diff'] = (agg_trans['volume_last'] - agg_trans['volume_mean']) / agg_trans['volume_mean']
agg_trans['forms_max_last_diff'] = (agg_trans['forms_last'] - agg_trans['forms_max']) / agg_trans['forms_max']
agg_trans['forms_mean_last_diff'] = (agg_trans['forms_last'] - agg_trans['forms_mean']) / agg_trans['forms_mean']

In [25]:
def get_vol_perc_gt_mean(org):
    this_org_df = trans[trans['org']==org]
    this_mean = this_org_df[this_org_df['volume']>0]['volume'].mean()
    return float(len(this_org_df[this_org_df['volume']>this_mean])) / float(len(this_org_df))

def get_forms_perc_gt_mean(org):
    this_org_df = trans[trans['org']==org]
    this_mean = this_org_df[this_org_df['forms']>0]['forms'].mean()
    return float(len(this_org_df[this_org_df['forms']>this_mean])) / float(len(this_org_df))

agg_trans['volume_perc_gt_mean'] = agg_trans['org'].apply(get_vol_perc_gt_mean)
agg_trans['forms_perc_gt_mean'] = agg_trans['org'].apply(get_forms_perc_gt_mean)

## analytics

In [26]:
q = "select * from analytics_month where date>=2018"
df_base = redshift_query_read(q)
q = "select * from analyticsqgiv_month where date>=2018"
df_qgiv = redshift_query_read(q)

In [27]:
df_analytics = df_base.merge(df_qgiv, on=["org", "form", "date"]).dropna()

df_analytics = df_analytics.drop(['form', 'product'], axis=1).groupby(['date', 'org']).sum().reset_index()
df_analytics['date'] = pd.to_datetime(df_analytics['date'])

agg_analytics = None
for org in df_analytics['org'].unique():
    this_df = df_analytics[df_analytics['org']==org].copy()
    last_3_months = this_df['date'].sort_values(ascending=True).tail(4)
    this_df[this_df['date'].isin(last_3_months)]
    
    pct_change = this_df.drop(['date', 'org'], axis=1).pct_change().mean().reset_index().transpose()
    pct_change.columns = pct_change.iloc[0]
    pct_change = pct_change[1:]
    pct_change['org'] = org
    
    if agg_analytics is None:
        agg_analytics = pct_change
    else:
        agg_analytics = agg_analytics.append(pct_change)

In [28]:
agg_analytics = agg_analytics.fillna(0).replace(np.inf, 100.).replace(-np.inf, -100.)
agg_analytics.head()

index,vt_trans_count,don_form_trans_count,kiosk_trans_count,p2p_trans_count,mobile_trans_count,mobilevt_trans_count,sms_trans_count,fb_trans_count,vt_trans_vol,don_form_trans_vol,...,enable_donorlogins,enable_sms,new_rec_volume,new_rec_count,reg_count,dl_trans_volume,dl_trans_count,dl_new_rec_count,dl_new_rec_volume,org
0,0.004618,0.059776,0.0,0,0.213113,100.0,100.0,0.0,0.010111,0.203808,...,100.000000,0.090909,100.0,100.0,0.214499,0.220079,0.112429,0.0,0.0,6
0,0.000000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,-0.012987,0.004545,100.0,100.0,100.000000,0.000000,0.000000,100.0,100.0,9
0,100.000000,0.370262,0.0,0,100.000000,100.0,100.0,0.0,100.000000,0.656808,...,0.000000,0.000000,100.0,100.0,100.000000,100.000000,100.000000,100.0,100.0,13
0,0.000000,0.464669,0.0,0,0.000000,100.0,100.0,100.0,0.000000,1.047172,...,100.000000,100.000000,100.0,100.0,100.000000,0.000000,0.000000,0.0,0.0,31
0,0.000000,0.651555,0.0,0,100.000000,0.0,0.0,0.0,0.000000,2.756539,...,0.000000,0.000000,100.0,100.0,0.000000,100.000000,100.000000,100.0,100.0,33


# feature set building

In [47]:
logs_pvt['org'] = logs_pvt['org'].astype(int)
login_change['org'] = login_change['org'].astype(int)
integrations_change['org'] = integrations_change['org'].astype(int)
cms_change['org'] = cms_change['org'].astype(int)
agg_trans['org'] = agg_trans['org'].astype(int)
agg_analytics['org'] = agg_analytics['org'].astype(int)

len(logs_pvt), len(login_change), len(integrations_change), len(cms_change), len(agg_trans), len(agg_analytics)

(5125, 2885, 52, 2545, 5336, 4054)

In [48]:
# build data set
target = orgs[['id', 'churned']]
target.columns = ['org', 'churned']

data = target.merge(logs_pvt, on="org", how="outer").merge(login_change, on="org", how="outer")
data = data.merge(integrations_change, on="org", how="outer").merge(cms_change, on="org", how="outer")
data = data.merge(agg_trans, on="org", how="outer").merge(agg_analytics, on="org", how="outer")

data = data.fillna(0)
data['churned'] = data['churned'].astype('int')
data['recent_login'] = data['recent_login'].apply(lambda x: False if x == 0 else x)

In [49]:
len(data.columns), len(data)

(105, 8484)

In [50]:
data.head(3)

,org,churned,system_0,system_2,system_4,system_8,system_11,system_12,system_13,system_15,...,collect_address_mobile,enable_donorlogins,enable_sms,new_rec_volume,new_rec_count,reg_count,dl_trans_volume,dl_trans_count,dl_new_rec_count,dl_new_rec_volume
0,441789,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,443044,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1045,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(['churned', 'org'], axis=1), data['churned'], test_size=.25)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

print(rf.score(X_test, y_test))

0.901933050447902


In [52]:
len(X_train), len(X_test)

(6363, 2121)

In [53]:
len_train_pos = len(y_train[y_train==1])
len_train_neg = len(y_train[y_train==0])
len_test_pos = len(y_test[y_test==1])
len_test_neg = len(y_test[y_test==0])

print("training: {} positives; {} negatives".format(len_train_pos, len_train_neg))
print("testing: {} positives; {} negatives".format(len_test_pos, len_test_neg))

training: 1299 positives; 5064 negatives
testing: 440 positives; 1681 negatives
